# Moleküler Dinamik Simülasyonları Bileşenleri
[EpiSTEM Türkiye'deki](https://www.epistemturkiye.org) yazıyı takiben biraz daha detaya girelim nedir bu simulasyonlar. Bileşenleri nelerdir? 

[Yazımda](epistem linki) da değindiğim gibi Newton'un kanunlarından yola çıkılmıştır moleküler dinamik simülasyonları için. Bir cismin hareketini ona uygulanan kuvvet ile orantılıdır. Elinizle bir objeyi ne kadar uzağa fırlatabileceğiniz ona uyguladığınız kuvvet ile belirlenir. Moleküler dinamik simülasyonlarında da bir molekülün hareketide ona uygulanan kuvvete bağlıdır. Bu kuvveti tanımlarken molekülün sahip olduğu geometrik yapıyı ve çevresini saran diğer molekülleri baz alıyoruz. İşte bu aşamada kuvvet alanı dediğimiz bir grup özellikler devreye giriyor. 
Nasıl insanların kol uzunlukları farklı ise, atomları birbine bağlayan bağlarda farklı uzunluklara sahipler. Kimimizin kolu çabuk kırılırken, kimimiz sağlam. Bu bağların dayanıklılığı da aynı böyle değişkendir. Meraklısına aşağıdaki grafik bir kuvvet alanındaki tanımlı özellikleri özetliyor. Ama ben daha fazla detaya girmeyeceğim. 
<img src="potansiyel_enerji.png"> (Kaynak Erik LindahlPublished 2008 in Methods in molecular biology
DOI:10.1007/978-1-4939-1465-4_1) 

Kuvvet alanı moleküler dinamik simülasyonlarının en temel ve en önemli bileşenleridir. Kuvvet alanına ek olarak bilmemiz gereken bir diğer bileşen ise koordinat. Yani atomların adresleri. Hepimizin bir ev ve iş adresi var. Koordinatlarıda atomların yerini belirleyen adresler olarak düşünebilirsiniz. Bu koordinatlar elbette 3 boyutlu. Basit moleküllerin koordinatlarını tahmin yolu ile çizmek mümkün olabilir. Fakat 5 tane yapı taşından (amino asitten) oluşan proteinin bile alabileceği çok sayıda şekil var. O nedenle deneysel yöntemlere başvuruyoruz bu koordinatları elde ederken.
İhtiyacımız olan üçüncü bileşen ise simülasyonu çalıştıracağımız program. Benim deneyimlerime göre GROMACS simülasyonlara yeni başlayanlar için en güvenilir program. Komut isteminden kullanmak bir çok kişiyi ürkütse de yeni başlayan biri olarak yapacağınız hataların geri bildirimini almak çok önemli ve GROMACS bu konuda en iyilerden biri diyebilirim. Kullanabileceğiniz diğer alternatifler NAMD, ki bunu VMD programından bir arayüz ile kullanabilirsiniz. Görsel açıdan öğrenmesi çok daha kolay bir program. CHARMM bir çok özelliği barındıran bir nevi kodlama dili, simülasyon yapabilirsiniz ama öğrenmesi kolay olmayan bir program (hata mesajlarının yoksunluğu da cabası). Ama GROMACS'a kıyasla sistemler üzerinde değişiklik yapması daha kolay bir program. 
Amber üzerine bir fikrim olmayan bir program, kullanmadım. Lakin aynı isimli kuvvet alanını kullandım ki bu rehberde onun üzerine. Alternatıf kuvvet alanı arıyorsanız, CHARMM isimli (yukarıda bahsettiğim programı geliştirenler tarafından hazırlanan) kuvvet alanına bakabilirsiniz.
Programı belirledik, GROMACS. Kuvvet alanımız, Amber. Kaldı geriye koordinatlar... Bunun için [PDB veri bankasına](http://www.rcsb.org/) gidip bir yapı indirdim: [İnsülin](http://www.ebi.ac.uk/pdbe/quips?story=insulin). Linkten daha fazla bilgi edinebilirsiniz. Bu proteini seçmemin sebebi, muhtemelen bir çoğunuzun ismini duymuş olması. Şeker hastası var mı aramızda? Kendinize enjekte etmek olduğunuz insülin ve insülin benzeri proteinler sayesinde hayat kaliteniz biraz daha artıyor, artık kendisi ile moleküler düzeyde de tanışmış oldunuz.

In [68]:
import nglview #NGLview paketini ortama yükle (https://github.com/arose/nglview)
view = nglview.show_pdbid("2ins")  #"2ins" kodlu yapıyı PDB veri bankasından indir ve NGLview uygulamasına gönder
view #görüntüle

NGLWidget()

Yukarıda görmüş olduğunuz yapı insülinin aktif formu. Proteinlerin aktif formları birbirinin aynı veya farklı bir çok zincirden oluşabilir. Hız kazanmak açısından bu rehberde biz sadece aşağıda göreceğiniz tek bir zincir ile devam edeceğiz. Bu arada üstteki resim aslında hareketli, üzerine imleci getirdiğinizde size yapıtaşlarının bilgisini veriyor: Formatı ise şöyle atom: [amino asit ismi]numarası:ait olduğu zincir:atom ismi. Proteini üzerine tıklayıp döndürebilir, kontrol tuşuna basılı tutup sağa sola sürükleyebilirsiniz. Yakınlaştırıp uzaklaştırabilirsiniz.

## 1. Simülasyon ortamının hazırlanması
### a. Protein yapısının temizlenmesi
Her ne kadar sizi burada angarya dediğim bu kısım ile uğraştırmayacak olsam da bilmeniz gerek. Protein veri bankasından indirip de öyle hemen simülasyon başlatabileceğiniz yapı sayısı azdır. Bunun en birincil nedeni deneysel yöntemlerin her atoma konum atamakta eksik kalmasından kaynaklı. Günümüzde detayına inmediğim bu yöntemler (X-ray, NMR... vs) daha yüksek kalitede protein resimleri çizebiliyor olsa da proteinin her bölgesi görüntülenemeyebiliyor. İkincil neden proteinin ile beraber başka moleküllerinden resminin çıkması ve bizim bunları simülasyonda istemiyor olmamız. Bu moleküllerden bazıları deneyin başarılı olması için gerekli (mesela deterjanlar), bazıları da proteinlere bağlanan ilaç gibi diğer bileşenlerdir. Genelde deterjan gibi bileşenleri simülasyonda istemiyoruz, çünkü proteinin doğal ortamında aslında onlar yok. İlaç benzeri molekülleri ise kuvvet alanında tanımları olmadığı için çıkarmanız gerekebilir. Diğer nedenler ki bunlar aslında ikincil nedenden daha fazla sıklıkta görülüyor, protein numaralandırılmasının farklı olması, bazı bağların eksık olması, amino asitlerin eksik olması ama eksik olarak rapor edilmemiş olması, vs. vs. Hayatınızı kolaylaştırmanın tek yolu bir protein ailesi ile çalışmaya başlamadan önce numaralandırma ve doğal amino asit dizisi kontrollerini yapıp aklı selim bir rota belirlemeniz.
Ne çok laf ettin diyeceksiniz, inanın bu yapıları hazırlamak simülasyonların çalışmasından daha uzun sürüyor. Bu aşamada bir hata yapmış iseniz... hoooop... bütün veriler çöpe.
Ben size aşağıda, buyurun, hazır temizlenmişini sunuyorum.

In [69]:
import nglview #bu komut üstte çalıştığı için tekrar çalıştırmaya gerek yok ama üstteki veri kayboldu ise tekrar yükledim.
view = nglview.show_file("2ins_zincirAB.pdb")  #"2ins" yapısının A ve B zincirne ait dosyayı NGLview uygulamasına gönder
view #görüntüle

NGLWidget()

### b. Protein yapısının kuvvet alanı ile hazırlanması
Fark ettiniz mi hala iki zincir var insülinde. Aslında tek bir peptid zincirinden oluşur insülin, yani A ve B zincirleri tek bir gen tarafından üretilmiştir, ancak insülün aktif halini almadan önce bazı değişimlerden geçer ve sonucunda sanki birbirinden bağımsızmış gibi iki zincir oluşur.
Biz şimdi bu iki zinciri kuvvet alanındaki tanımları ile "topoloji" denilen bir dosyaya yeniden yazacağız. PDB gibi dosyalar proteindeki atomların uzaydaki yerini belirlerken, topoloji bize bu atomların birbirleri ve çevresi ile ilişkisi hakkında nicel bilgiler içerir. Karbon atomu, oksijenden 0.15 nanometre uzakta (evet geçiniz kilometreyi santimetreyi biz nano boyuttayız artık) ve oksijen dışında 1'i hidrojen olmak üzere 3 atoma daha bağlı. Karbon-hidrojen-oksijen açısı 120 derece. ... Bu şekilde bir sürü bilgi var, bunların el ile hazırlanması mümkün olsa da, aklımızı peynir ekmek ile yemedik bu devirde. Sağ olsunlar simülasyon programlarını yazanlar bize bu dosyaları da hazırlayacak bir bileşen eklemişler. Aşağıdaki komut ile bu dosyayı çok kolay hazırlayabiliriz. (Bu komutları bash gibi bir terminalde doğrudan çalıştırıyorum).

In [70]:
%%bash
export GMX_MAXBACKUP=-1
pdb2gmx -f 2ins_zincirAB.pdb -o kuvvetli_protein.pdb -p topoloji.top -ff amber99sb-ildn -water tip3p -merge all



Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

Reading 2ins_zincirAB.pdb...
Read 393 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.

Merged chains into joint molecule definitions at 1 places.

There are 1 chains and 0 blocks of water and 30 residues with 393 atoms

  chain  #res #atoms
  1 'A'    50    393  

Reading residue database... (amber99sb-ildn)
Processing chain 1 'A' (393 atoms, 50 residues)
Identified residue GLY1 as a starting terminus.
Identified residue ASN21 as a ending terminus.
Identified residue VAL2 as a starting terminus.
Identified residue ALA30 as a ending terminus.
Checking for duplicate atoms....
Now there are 389 atoms. Deleted 4 duplicates.
Generating any missing hydrogen atoms and/or adding termini.
Now there are 50 residues with 755 atoms
		--------- PLEASE NOTE ------------
You have successfully generated a topology from: 2ins_zincirAB.pdb.
The Amber99sb-ildn force field and the tip3p wate

                         :-)  G  R  O  M  A  C  S  (-:

                   Great Red Oystrich Makes All Chemists Sane

                            :-)  VERSION 4.6.5  (-:

        Contributions from Mark Abraham, Emile Apol, Rossen Apostolov, 
           Herman J.C. Berendsen, Aldert van Buuren, Pär Bjelkmar,  
     Rudi van Drunen, Anton Feenstra, Gerrit Groenhof, Christoph Junghans, 
        Peter Kasson, Carsten Kutzner, Per Larsson, Pieter Meulenhoff, 
           Teemu Murtola, Szilard Pall, Sander Pronk, Roland Schulz, 
                Michael Shirts, Alfons Sijbers, Peter Tieleman,

               Berk Hess, David van der Spoel, and Erik Lindahl.

       Copyright (c) 1991-2000, University of Groningen, The Netherlands.
         Copyright (c) 2001-2012,2013, The GROMACS development team at
        Uppsala University & The Royal Institute of Technology, Sweden.
            check out http://www.gromacs.org for more information.

         This program is free software; you can redis

Bu komutta -f ile girdiyi, -o ile çıktıyı tanımladık (bu atomları kuvvet alanına göre isimlendirilmiş bir dosya). -p topoloji dosyasını tanımladı, -ff kuvvet alanını, -water su modelini seçti (anlaşılması en zor molekül sudur, birden fazla modeli vardır). -merge all ise iki zincirin topoloji bilgisini tek bir dosyaya (topoloji.top) yazdı. Üstteki seyir defterini (log) incelerseniz, insülinin toplam yükünü öğrenebilirsiniz. 

### c. Simülasyon kutusu ve çözelti
Simülasyonları bir deney tüpü olarak düşünebilirsiniz. Sadece bir sürü protein ile aynı anda deney yapmak yerine biz her bir proteini tek tek tüplere yerleştiriyoruz. Bu tüperin sınırını biz çiziyoruz. Gönül isterdi ki kocaman hazırlayalım bu simülasyonları ama çok pahalı bir hesaplama olduğu için mümkün olduğunca ufak hazırlıyoruz. Aşağıdaki komutlar önce kutunun sınırlarını çizip, sonra da o kutuyu su ile dolduracak. Bir önceki aşamada seçtiğimiz su modeli de burada devreye giriyor. Kutunun büyüklüğü proteinin etrafında yaklaşık 1 nanometre boşluk kalacak şekilde 5.4 nanometre olarak seçildi. 
Su ekledikten sonra ise sistemin toplam yükünü sıfırlıyor (1) ve belirli miktarda bir çözelti hazırlıyoruz (2).

1) Atomlar arasındaki etkileşimi hesaplamada kullanılan yöntem sistemin toplam yükünün sıfır olmasını gerektiriyor. Bu yük dengeleme işi sisteme iyonlar eklenerek yapılıyor. İnsülinin yükünü öğrendi iseniz sisteme artı yüklü sodyum mu yoksa eksi yüklü klorür mü eklememiz gerekiyor tahmin edebilirsiniz, çünkü kutuya eklenen suyun toplam yükü sıfır. Yani sistem insülinin yükünü taşıyor.

2) İnsülinin içinde bulunduğu doğal ortamı sistemi mümkün olduğunca basit tutarak hazırlıyoruz. Bunun en temel yolu iste sisteme daha fazla iyon yüklemek. Yani sadece net yükü sıfırlamakla kalmıyoruz, üzerine daha fazla sayıda sodyum ve klorür ekliyoruz.

-p topoloji.top bu dosyasının devamlı güncellenmesini sağlıyor, böylece sistemde ekleme/çıkarmalar sonucunda ne kaldı biliyoruz. Bu işlem önemli çünkü topoloji dosyasının PDB dosyası ile uyuşması gerek.

In [71]:
%%bash
export GMX_MAXBACKUP=-1
editconf -f kuvvetli_protein.pdb -box 5.4 5.4 5.4 -bt cubic -o kutuda_protein.pdb
genbox -cp kutuda_protein.pdb -p topoloji.top -cs -o sulu_protein.pdb

Read 755 atoms
No velocities found
    system size :  2.478  2.554  3.217 (nm)
    diameter    :  3.356               (nm)
    center      : -0.996  1.065  0.534 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  3.696  1.635  2.166 (nm)
new center      :  2.700  2.700  2.700 (nm)
new box vectors :  5.400  5.400  5.400 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  : 157.46               (nm^3)

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

Neighborsearching with a cut-off of 0.48
Table routines are used for coulomb: FALSE
Table routines are used for vdw:     FALSE
Cut-off's:   NS: 0.48   Coulomb: 0.48   LJ: 0.48


                         :-)  G  R  O  M  A  C  S  (-:

               Gromacs Runs One Microsecond At Cannonball Speeds

                            :-)  VERSION 4.6.5  (-:

        Contributions from Mark Abraham, Emile Apol, Rossen Apostolov, 
           Herman J.C. Berendsen, Aldert van Buuren, Pär Bjelkmar,  
     Rudi van Drunen, Anton Feenstra, Gerrit Groenhof, Christoph Junghans, 
        Peter Kasson, Carsten Kutzner, Per Larsson, Pieter Meulenhoff, 
           Teemu Murtola, Szilard Pall, Sander Pronk, Roland Schulz, 
                Michael Shirts, Alfons Sijbers, Peter Tieleman,

               Berk Hess, David van der Spoel, and Erik Lindahl.

       Copyright (c) 1991-2000, University of Groningen, The Netherlands.
         Copyright (c) 2001-2012,2013, The GROMACS development team at
        Uppsala University & The Royal Institute of Technology, Sweden.
            check out http://www.gromacs.org for more information.

         This program is free software; you can re

İyonları eklemeden önce sisteme bakalım mı? Üstteki seyir defterinde kaç tane su molekülü eklenmiş bulabilirsiniz. Alttada görsel hali... 

In [72]:
import nglview #bu komut üstte çalıştığı için tekrar çalıştırmaya gerek yok ama üstteki veri kayboldu ise tekrar yükledim.
view = nglview.show_file("sulu_protein.pdb")
view.add_ball_and_stick("sol",color="cyan")
view #görüntüle

NGLWidget()

İyonları eklerken GROMACS'ın bir gerekliliği var, o da çalıştırma dosyası yaratmak. Bunun detaylarına girmiyorum, her simülasyon programının farklı bir yolu var bunun için. Ama aynı işlevi görüyorlar. Rastgele su moleküllerini iyonlar ile değiştirmek. Yani bu aşamada sistemden su çıkararak iyonlara yer açıyoruz. Eklediğim iyon çözeltisinin miktarını aşağıdaki komut isteminden tahmin edebilirsiniz (birim mol/litre).

In [73]:
%%bash
export GMX_MAXBACKUP=-1
touch 4ion.mdp
grompp -f 4ion.mdp -c sulu_protein.pdb -p topoloji.top -o 4ion.tpr
genion -s 4ion.tpr -conc 0.15 -neutral -o cok_iyonize_protein.pdb << EOF
13
EOF

Analysing residue names:
There are:    50    Protein residues
There are:  4924      Water residues
Analysing Protein...
Largest charge group radii for Van der Waals: 0.039, 0.039 nm
Largest charge group radii for Coulomb:       0.084, 0.084 nm
This run will generate roughly 1 Mb of data
Will try to add 16 NA ions and 14 CL ions.
Select a continuous group of solvent molecules
Selected 13: 'SOL'


                         :-)  G  R  O  M  A  C  S  (-:

                      GROup of MAchos and Cynical Suckers

                            :-)  VERSION 4.6.5  (-:

        Contributions from Mark Abraham, Emile Apol, Rossen Apostolov, 
           Herman J.C. Berendsen, Aldert van Buuren, Pär Bjelkmar,  
     Rudi van Drunen, Anton Feenstra, Gerrit Groenhof, Christoph Junghans, 
        Peter Kasson, Carsten Kutzner, Per Larsson, Pieter Meulenhoff, 
           Teemu Murtola, Szilard Pall, Sander Pronk, Roland Schulz, 
                Michael Shirts, Alfons Sijbers, Peter Tieleman,

               Berk Hess, David van der Spoel, and Erik Lindahl.

       Copyright (c) 1991-2000, University of Groningen, The Netherlands.
         Copyright (c) 2001-2012,2013, The GROMACS development team at
        Uppsala University & The Royal Institute of Technology, Sweden.
            check out http://www.gromacs.org for more information.

         This program is free software; you can redistrib

In [74]:
import nglview #bu komut üstte çalıştığı için tekrar çalıştırmaya gerek yok ama üstteki veri kayboldu ise tekrar yükledim.
view = nglview.show_file("cok_iyonize_protein.pdb")
view.add_surface("sol and not within 3 of protein",color="cyan",opacity=0.1)
view.add_ball_and_stick("NA",radiusSize=0.5) #mor renkli
view.add_ball_and_stick("CL",radiusSize=0.5) # yeşil renkli. 
view #görüntüle

NGLWidget()

## 2.Simülasyon

Bu kısım için maalesef size bende hazır yapılmışı var diyeceğim. Ama size tabiki bu simülasyonu nasıl elde ettiğimi anlatacağım. Sağlıklı bir veri elde etmemiz için öncelikle sistemi biraz rahatlatmamız lazım. Biz proteini hazırlarken, suyu ve iyonları eklerken hiç sormadık acaba keyfiniz yerinde mi, komşu atomlarınız sizi çok rahatsız ediyor mu diye. 
Bu aşamada sistemdeki her bir bileşenin hem kendi içindeki hem de çevresindeki kavgalarını çözeceğiz. Bir diğer deyişle, enerjisini azaltacağız. İki ayrı aşamada yapılıyor; ilkinde her molekül bulunduğu yerde kıpraştırılıyor (aynen telefonun kıpraşması gibi bu, oldukları yerde hafifçe oynuyorlar), ikincisinde ise protein oynamasın diye üzerine güç uygulayıp su ve iyonlara hadi nerede mutlu iseniz oraya doğru yol alın diyoruz. Bu ikinnci aşama aslında moleküler dinamik simülasyonun ta kendisi. Sadece cihazları kalibre etmek gibi bir amacı olduğu için bu simülasyonun, genelde biz bu verileri incelemiyoruz.
Veri toplanmasını her molekülün serbest dolaşım hakkına sahip olduğu üretim simülasyonlarından yapıyoruz. Bu simülasyonda vize yok, her şey serbest. İşte size aşağıda bu simülasyonu gösteriyorum. Pek bir şey olmadığını göreceksiniz, malum ancak 1-2 nanosaniye uzunlukta.Gülün gülün, bende gülüyorum, o bir iki nanosaniyeyi alabilmek için eskiden kaç yıllar geçerdi. Bir deney tüpünde var 1 milyon protein bense gitmiş tek bir tane proteini bir kaç nanosaniye kıpraştıracağım diye debeleniyorum. 
Öyle bir delilik hali işte bilim. Ama yine de güzel değil mi, insülün ile tanışıp onu sulu bir ortama yerleştirdiniz. İleride simülasyon kısmını detaylandıracağım bir rehberde de koşullar (misal sıcaklık) ile oynarak neler yapabilirizi öğreniriz. Bu alana ilk başladığımda ben böyle izliyordum bu videoları, keşif yapıyordum. Sonra öğrendim ki meğer grafikler alınıyormuş, her şey matematik her şey istatistikmiş. Şimdi o üstteki resimlere ancak makale yazacağım zaman bakıyorum ben. 
Neyse uzun bir rehber oldu ama umarım eğlenmişsinizdir. Bilim ve sanatla kalın! Hoşçakalın. 